# Foxglove Data Platform Demo Notebook

This notebook demonstrates retrieving data and events from Foxglove Data Platform.


In [ ]:
# Load our API key from our .env file

from dotenv import dotenv_values

config = dotenv_values(".env")
api_key = config["API_KEY"]
api_key

In [ ]:
# Now we can create an instance of our client.

from foxglove.data_platform.client import Client, OutputFormat

client = Client(token=api_key)

In [ ]:
# Next let's use the client to list some events.

events = client.list_events(
    device_name="Milesbot", limit=5, key="position", value="start"
)

In [ ]:
# Now we can download some data from data platform.

import sys
from datetime import datetime, timedelta

start_event = events[-1]
start_time = start_event["timestamp"]
end_time = start_time + timedelta(seconds=60 * 10)
device_id = start_event["device_id"]

download_stream = client.download_data(
    device_id=device_id,
    start=start_time,
    end=end_time,
    output_format=OutputFormat.mcap0,
    topics=["/gps"],
)

data = bytes()
for chunk in download_stream.iter_content(chunk_size=64 * 1024):
    data += chunk
len(data)

In [ ]:
# Let's read some messages from our mcap stream.

from io import BytesIO

import pandas
from mcap.mcap0.stream_reader import StreamReader
from mcap_ros1.ros1_decoder import Ros1Decoder

reader = StreamReader(BytesIO(data))
decoder = Ros1Decoder(reader)
messages = [
    (message.latitude, message.longitude) for topic, message in decoder.messages
]
pandas.DataFrame(messages[0:3], columns=["lat", "lon"])

In [ ]:
import folium

map = folium.Map(location=messages[0], zoom_start=200, width="50%")
folium.PolyLine(
    locations=messages,
    weight=10,
    color="orange",
).add_to(map)
map